In [176]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [177]:
im = cv2.imread('./images/prtn00.jpg', cv2.IMREAD_COLOR)
im_gray = cv2.imread('./images/prtn00.jpg', cv2.IMREAD_GRAYSCALE)

In [178]:
def get_gauss_pyramid(im, l=4, sub=2, smoothing=1.0):
    res = [im]
    
    for i in range(l):
        tmp = cv2.GaussianBlur(im, (0,0), smoothing)
        tmp = cv2.resize(im, (0,0), fx=1/sub, fy=1/sub)
        res.append(tmp)
        im = tmp
    
    return res

In [179]:
# def get_harris_matrix(im, si=1.5, sd=1):
#     h, w = im.shape[0], im.shape[1]
#     H = np.zeros((h, w))
    
#     for i in range(h):
#         for j in range(w):
#             H[i, j] = cv2.Sobel(im, -1, sd, sd)

In [235]:
def get_harris_matrix(im, si=1.5, sd=1):
    h, w = im.shape[0], im.shape[1]
    prod = []
    
    dx = cv2.Sobel(im, -1, sd, 0)
    dy = cv2.Sobel(im, -1, 0, 1)
    
    for i in range(h):
        p_tmp = []
        for j in range(w):
            tmp = np.array([[dx[i, j]], [dy[i, j]]])
            tmp_prod = np.matmul(tmp, np.transpose(tmp))
            tmp_prod = cv2.GaussianBlur(tmp_prod, (0,0), si)
            p_tmp.append(tmp_prod)
    
        prod.append(p_tmp)
    
    prod = np.array(prod)
        
    return prod

In [236]:
# def get_harris_matrix(im, si=1.5, sd=1):
#     d = cv2.Sobel(im, -1, sd, sd)
#     dT = np.transpose(d)
#     prod = np.matmul(d, dT)
#     H = cv2.GaussianBlur(prod, (0,0), si)
    
#     return H

In [264]:
def get_corner_strength(H):
    res = []
    
    for i in range(H.shape[0]):
        tmp = []
        for j in range(H.shape[1]):
            det = np.linalg.det(H[i, j])
            trace = np.trace(H[i, j])
            if trace == 0:
                tmp.append(0)
            else:
                tmp.append(det/trace)
#             w, v = np.linalg.eig(H[i, j])
#             eig1 = w[0]
#             eig2 = w[1]
# #             if i == 10 and j == 10:
#             print(eig1)
            
#             if (eig1+eig2) == 0:
#                 tmp.append(0)
#             else:
#                 tmp.append((eig1*eig2)/(eig1+eig2))
        res.append(tmp)
        
    return np.array(res)

In [258]:
blurred_images = get_gauss_pyramid(im_gray)

In [246]:
H = get_harris_matrix(blurred_images[0])

In [265]:
fhm = get_corner_strength(H)

In [266]:
f = fhm.flatten()

In [267]:
max(f)

1.421085471520205e-14